In [15]:
!pip install git+https://github.com/ContinualAI/avalanche.git

  Cloning https://github.com/ContinualAI/avalanche.git to /tmp/pip-req-build-_3l8aj0c
  Running command git clone -q https://github.com/ContinualAI/avalanche.git /tmp/pip-req-build-_3l8aj0c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


# necessary imports - omniglot data

In [16]:
import torch
import torchvision
from torchvision import transforms

from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitOmniglot
from avalanche.benchmarks.datasets.omniglot import Omniglot
from avalanche.benchmarks.classic import SplitMNIST
from avalanche.evaluation.metrics import forgetting_metrics, accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, confusion_matrix_metrics, disk_usage_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.strategies import Naive

# benchmark and model for omniglot

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
height = 32
n_classes = 964
batch_size = 8
epochs = 5

benchmark = SplitOmniglot(n_experiences=4, 
                          seed=1, 
                          train_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]), 
                          eval_transform=transforms.Compose([
                              transforms.Resize(height),
                              transforms.Grayscale(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.9221,), (0.2681,))
                          ]))

model = SimpleMLP(num_classes=n_classes, input_size=height*height)

optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=batch_size, train_epochs=epochs,
    eval_mb_size=batch_size, device=device)

print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start of experience: ", experience.current_experience)
    print("Current Classes: ", experience.classes_in_this_experience)

    cl_strategy.train(experience)
    print('Training completed')
    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(benchmark.test_stream))

Files already downloaded and verified
Files already downloaded and verified
Starting experiment...
Start of experience:  0
Current Classes:  [2, 8, 520, 10, 529, 531, 534, 28, 541, 542, 33, 35, 548, 550, 40, 554, 556, 47, 559, 561, 564, 567, 568, 58, 571, 573, 62, 574, 64, 580, 70, 72, 74, 589, 78, 590, 592, 593, 594, 595, 598, 87, 600, 90, 93, 607, 97, 100, 102, 614, 617, 107, 623, 626, 115, 628, 122, 123, 636, 126, 638, 129, 647, 139, 651, 653, 143, 144, 145, 146, 655, 148, 656, 657, 659, 152, 664, 668, 157, 161, 163, 676, 166, 679, 168, 681, 171, 685, 174, 686, 693, 696, 698, 187, 699, 191, 705, 706, 195, 196, 707, 708, 199, 200, 710, 202, 716, 721, 211, 212, 213, 728, 729, 218, 733, 223, 738, 739, 228, 229, 233, 241, 756, 761, 762, 763, 252, 253, 764, 255, 766, 770, 774, 775, 264, 778, 267, 269, 271, 784, 786, 789, 791, 283, 284, 796, 287, 289, 801, 804, 805, 295, 809, 812, 813, 308, 309, 310, 821, 312, 827, 829, 318, 833, 322, 835, 325, 837, 327, 840, 841, 330, 331, 847, 337, 849,

In [19]:
from avalanche.benchmarks.utils import ImageFolder, DatasetFolder, FilelistDataset, AvalancheDataset
dataset = torchvision.datasets.Omniglot(
    root="./data", download=True, transform=torchvision.transforms.ToTensor()
)

image, label = dataset[0]
print(type(image))  # torch.Tensor
print(type(label))  # int
height = 32
TRANSFORM_IMG = transforms.Compose([
  transforms.Resize(height),
  transforms.Grayscale(),
  transforms.ToTensor()
])

ds = ImageFolder('./data/omniglot-py/images_background/Korean', transform=TRANSFORM_IMG)
print(ds)

Files already downloaded and verified
<class 'torch.Tensor'>
<class 'int'>
Dataset ImageFolder
    Number of datapoints: 800
    Root location: ./data/omniglot-py/images_background/Korean
    StandardTransform
Transform: Compose(
               Resize(size=32, interpolation=bilinear, max_size=None, antialias=None)
               Grayscale(num_output_channels=1)
               ToTensor()
           )


# Evaluation

In [21]:
from avalanche.benchmarks.scenarios.new_classes.nc_scenario import NCScenario
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
n_classes = 40
batch_size = 8
epochs = 5
n_experiences = 20

model = SimpleMLP(num_classes=n_classes, input_size=height*height)

scenario = NCScenario(ds, ds, n_experiences=n_experiences, task_labels=n_classes)
train_stream = scenario.train_stream
test_stream = scenario.test_stream

optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=batch_size, train_epochs=epochs,
    eval_mb_size=batch_size, device=device)

results = []
for train_task in train_stream:
    print("Start of experience: ", train_task.current_experience)
    print("Current Classes: ", train_task.classes_in_this_experience)
    print("Classes seen so far: ", train_task.classes_seen_so_far)
    print()

    cl_strategy.train(train_task)
    print('Training completed')

    print('Computing accuracy on the whole test set')
    results.append(cl_strategy.eval(test_stream))

Start of experience:  0
Current Classes:  [34, 39]
Classes seen so far:  [34, 39]

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 5/5 [00:00<00:00, 100.24it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 3.4532
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.2250
100%|██████████| 5/5 [00:00<00:00, 98.67it/s] 
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.8771
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4750
100%|██████████| 5/5 [00:00<00:00, 91.66it/s]
Epoch 2 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.9189
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.5000
100%|██████████| 5/5 [00:00<00:00, 88.66it/s]
Epoch 3 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7454
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.4500
100%|██████████| 5/5 [00:00<00:00, 97.42it/s]
Epoch 4 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.7130
	Top1